In [428]:
from pathlib import Path
import numpy as np
from tqdm import tqdm

In [429]:
input = Path("example.txt").read_text()
print(input)

....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...



In [430]:
GUARD = "^"
OBSTACLE = "#"
UP = np.array([-1, 0])
RIGHT = np.array([0, 1])
DOWN = np.array([1, 0])
LEFT = np.array([0, -1])
DIRECTIONS = (UP, RIGHT, DOWN, LEFT)

In [431]:
start_arr = np.array([[char for char in line] for line in input.splitlines()])
print(start_arr)

[['.' '.' '.' '.' '#' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '#']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '#' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '#' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '#' '.' '.' '^' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '#' '.']
 ['#' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '#' '.' '.' '.']]


In [432]:
start_position = tuple(np.argwhere(start_arr == GUARD)[0])
print(start_position)

(6, 4)


In [433]:
obstacle_positions = np.argwhere(start_arr == OBSTACLE)
print(obstacle_positions)

[[0 4]
 [1 9]
 [3 2]
 [4 7]
 [6 1]
 [7 8]
 [8 0]
 [9 6]]


In [434]:
# Solution 1
direction_index = 0
direction = UP
visited_arr = np.zeros_like(start_arr, dtype=bool)
next = start_position
position = next

while (
    next[0] >= 0
    and next[0] < visited_arr.shape[0]
    and next[1] >= 0
    and next[1] < visited_arr.shape[1]
):
    visited_arr[tuple(position)] = True
    next = position + direction
    if (next == obstacle_positions).all(axis=1).any():
        direction_index = (direction_index + 1) % 4
        direction = DIRECTIONS[direction_index]
        continue
    position = next
print(visited_arr.astype(int))

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 1 1 1 0]
 [0 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 1 0 0 0 1 0]
 [0 0 1 1 1 1 1 0 1 0]
 [0 0 1 0 1 0 1 0 1 0]
 [0 0 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 0 0]
 [0 1 1 1 1 1 1 1 0 0]
 [0 0 0 0 0 0 0 1 0 0]]


In [435]:
# Solution 2
potential_obstacles = np.argwhere(visited_arr)
potential_obstacles[(potential_obstacles != start_position).all(axis=1)]
confirmed_obstacles = []
for obstacle in tqdm(potential_obstacles):
    direction_index = 0
    direction = UP
    visited_direction_arr = np.zeros(
        (4, start_arr.shape[0], start_arr.shape[0]),
        dtype=bool,
    )
    next = start_position
    position = next
    temp_obstacle_positions = np.append(
        obstacle_positions,
        np.array([obstacle]),
        axis=0,
    )

    cycle = False
    while (
        position[0] >= 0
        and position[0] < start_arr.shape[0]
        and position[1] >= 0
        and position[1] < start_arr.shape[1]
    ):
        # print(visited_direction_arr)
        if visited_direction_arr[direction_index][tuple(position)]:
            cycle = True
            break
        visited_direction_arr[direction_index][tuple(position)] = True

        next = position + direction
        if (next == temp_obstacle_positions).all(axis=1).any():
            direction_index = (direction_index + 1) % 4
            direction = DIRECTIONS[direction_index]
            continue
        position = next

    if cycle:
        confirmed_obstacles.append(obstacle)
len(confirmed_obstacles)

100%|██████████| 41/41 [00:00<00:00, 2781.14it/s]


6